In [0]:
import keras
from keras.layers import Dense, Conv2D, BatchNormalization, Activation, Dropout
from keras.layers import AveragePooling2D, Input, Flatten
from keras.optimizers import Adam
from keras.callbacks import ModelCheckpoint, LearningRateScheduler
from keras.callbacks import ReduceLROnPlateau
from keras.preprocessing.image import ImageDataGenerator
from keras.utils import np_utils
from keras.regularizers import l2
from keras import backend as K
from keras.models import Model
from keras.datasets import cifar10
import numpy as np
import pickle
import os

(x_train,y_train),(x_test,y_test) = cifar10.load_data()
x_train, x_test = x_train.astype('float32')/255 , x_test.astype('float32')/255
mean = np.mean(x_train,axis=0)
std = np.std(x_train,axis=0)+1e-7
x_train,x_test = (x_train-mean)/std, (x_test-mean)/std

y_train = np_utils.to_categorical(y_train,10)
y_test = np_utils.to_categorical(y_test,10)


def set_lr(epoch):
    lr = 1e-3
    if epoch > 180: lr = 1e-7
    if epoch > 150: lr= 1e-6
    elif epoch > 120: lr = 1e-5
    elif epoch > 90:lr = 5e-5
    elif epoch> 60: lr= 5e-4
    elif epoch > 45: lr = 1e-4
    print('Learning rate: ', lr)
    return lr


def conv_layer(inputs,num_filters=16,kernel_size=3, strides=1, activation='elu', batch_norm=True):
    x = Conv2D(num_filters, kernel_size=kernel_size, strides=strides, padding='same', kernel_initializer='he_normal', kernel_regularizer=l2(1e-4))(inputs)
    if batch_norm: x = BatchNormalization()(x)
    if activation: x = Activation(activation)(x)
    return x


inputs = Input(shape=(32,32,3))
def resnet_layer(initial, filters =16,strides=1,first= False):
    out1 = conv_layer(initial, num_filters=filters,strides=strides)
    out1 = conv_layer(out1, num_filters=filters)
    out1 = conv_layer(out1, num_filters=filters, activation=None)
    if first: initial = conv_layer(initial,strides=strides,num_filters = filters,kernel_size=1,activation=None,batch_norm=False)

    output = keras.layers.add([initial, out1])
    output = Activation('elu')(output)
    return output

#### first Stack
initial = conv_layer(inputs)
O1 = resnet_layer(initial)
O1 = resnet_layer(O1)
O1 = resnet_layer(O1)
O1 = resnet_layer(O1)
O1 = resnet_layer(O1)

O1 = Dropout(0.2)(O1)
### Second stack
O2 = resnet_layer(initial=O1,filters=32,strides=2,first=True)
O2 = resnet_layer(initial=O2,filters=32)
O2 = resnet_layer(initial=O2,filters=32)
O2 = resnet_layer(initial=O2,filters=32)
O2 = resnet_layer(initial=O2,filters=32)

O2 = Dropout(0.2)(O2)
### third stack
O3 = resnet_layer(initial=O2,filters=64,strides=2,first=True)
O3 = resnet_layer(initial=O3,filters=64)
O3 = resnet_layer(initial=O3,filters=64)
O3 = resnet_layer(initial=O3,filters=64)
O3 = resnet_layer(initial=O3,filters=64)

O3 = Dropout(0.3)(O3)
avg = AveragePooling2D(pool_size=4)(O3)
f = Flatten()(avg)
outputs = Dense(10, activation='softmax', kernel_initializer='he_normal')(f)
model = Model(inputs=inputs, outputs=outputs)


model.compile(loss='categorical_crossentropy',
              optimizer=Adam(lr=set_lr(0)),
              metrics=['accuracy'])

model.summary()
lr_scheduler = LearningRateScheduler(set_lr)

lr_reducer = ReduceLROnPlateau(factor=np.sqrt(0.1),
                               cooldown=0,
                               patience=5,
                               min_lr=1e-7)

callbacks = [lr_reducer, lr_scheduler]

datagen = ImageDataGenerator(
    featurewise_center=False,
    samplewise_center=False,
    featurewise_std_normalization=False,
    samplewise_std_normalization=False,
    zca_whitening=False,
    rotation_range=15,
    width_shift_range=0.1,
    height_shift_range=0.1,
    horizontal_flip=True,
    vertical_flip=False
    )

datagen.fit(x_train)

    # Fit the model on the batches generated by datagen.flow().
hist = model.fit_generator(datagen.flow(x_train, y_train, batch_size=128),
                        validation_data=(x_test, y_test),
                        epochs=200, verbose=1, workers=4,
                        callbacks=callbacks)

# Score trained model.
scores = model.evaluate(x_test, y_test, verbose=1)
print('Test loss:', scores[0])
print('Test accuracy:', scores[1])

model_json = model.to_json()
with open("model_resnet_26.json", "w") as json_file:
    json_file.write(model_json)
model.save_weights("model_resnet_26.h5")



model_json = model.to_json()
with open("model_resnet_33.json", "w") as json_file:
    json_file.write(model_json)
model.save_weights("model_resnet_33.h5")

with open('info_33','wb') as f:
    pickle.dump(hist.history,f)

import matplotlib.pyplot as plt
import itertools

def plot_confusion_matrix(cm, classes,
                          normalize=False,
                          title='Confusion matrix',
                          cmap=plt.cm.summer):
    """
    This function prints and plots the confusion matrix.
    Normalization can be applied by setting `normalize=True`.
    """
    if normalize:
        cm = cm.astype('float') / cm.sum(axis=1)[:, np.newaxis]
        print("Normalized confusion matrix")
    else:
        print('Confusion matrix, without normalization')

    print(cm)

    plt.imshow(cm, interpolation='nearest', cmap=cmap,alpha=0.75)
    plt.title(title)
    plt.colorbar()
    tick_marks = np.arange(len(classes))
    plt.xticks(tick_marks, classes, rotation=60)
    plt.yticks(tick_marks, classes)

    fmt = '.2f' if normalize else 'd'
    thresh = cm.max() / 2.
    for i, j in itertools.product(range(cm.shape[0]), range(cm.shape[1])):
        plt.text(j, i, format(cm[i, j], fmt),
                 horizontalalignment="center",
                 color="black" if cm[i, j] > thresh else "black")

    plt.ylabel('True label')
    plt.xlabel('Predicted label')
    plt.tight_layout()


from sklearn.metrics import confusion_matrix


title = 'confunsion matrix of CNN classifier'
pred = np.argmax(model.predict(x_test),axis=1)
(x_train,y_train),(x_test,y_test) = cifar10.load_data()
conf_mt = confusion_matrix(y_test,pred)
np.set_printoptions(precision=2)
labels = ['airplane', 'automobile', 'bird', 'cat', 'deer', 'dog', 'frog', 'horse', 'ship', 'truck']

plt.figure(figsize=(10,10))
plot_confusion_matrix(conf_mt, classes=labels,
                      title='Confusion matrix for gray images')
plt.show()



Using TensorFlow backend.


170500096/170498071 [==============================] - 2s 0us/step











Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.

Learning rate:  0.001


Model: "model_1"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            (None, 32, 32, 3)    0                                            
__________________________________________________________________________________________________
conv2d_1 (Conv2D)               (None, 32, 32, 16)   448         input_1[0][0]                    
__________________________________________________________________________________________________
batch_normalization_1 (BatchNor (None, 32, 32, 16)   64          conv2d_1[0][0]                   
________________________________________________________________________

KeyboardInterrupt: ignored

In [0]:
hist.history

{'acc': [0.41801999999046324,
  0.56686,
  0.6294200000190735,
  0.668559999961853,
  0.6971399999809265,
  0.71942,
  0.7379200000190735,
  0.7513400000190735,
  0.7603199999618531,
  0.7708800000381469,
  0.77586,
  0.784,
  0.793560000038147,
  0.795339999961853,
  0.8007600000190734,
  0.80646,
  0.81446,
  0.8148000000190735,
  0.81906,
  0.824920000038147,
  0.8258999999809266,
  0.8282,
  0.83278,
  0.834800000038147,
  0.83628,
  0.8400999999809265,
  0.8430399999618531,
  0.8453800000190735,
  0.845379999961853,
  0.8485000000190734,
  0.849820000038147,
  0.8523599999809265,
  0.8537800000190735,
  0.85494,
  0.8581400000381469,
  0.8583599999618531,
  0.8616599999809265,
  0.8629400000190734,
  0.86346,
  0.8647,
  0.866100000038147,
  0.8678800000190735,
  0.8693399999618531,
  0.8683799999809265,
  0.871859999961853,
  0.873160000038147,
  0.9035599999809265,
  0.9166199999618531,
  0.9186999999809266,
  0.9218,
  0.9220600000190735,
  0.9244800000190735,
  0.9250799999618